In [ ]:
import numpy as np
import pickle

padded_features = np.load("../../data/processed_data/bezier_features_padded_improved.npy")
padded_target = np.load("../../data/processed_data/target_padded.npy")

with open("../../data/processed_data/alphabet", "rb") as f:
    alphabet = pickle.load(f)

print(padded_features.shape, padded_target.shape)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print(tf.config.list_physical_devices('GPU'))
print(tf.test.is_built_with_cuda())

In [ ]:
def split_data(features, target, train_size=0.9):
    size = len(features)
    indices = np.arange(size)
    np.random.shuffle(indices)
    train_samples = int(size * train_size)
    
    x_train, y_train = features[indices[:train_samples]], target[indices[:train_samples]]
    x_valid, y_valid = features[indices[train_samples:]], target[indices[train_samples:]]
    
    return (
        x_train,
        x_valid,
        y_train,
        y_valid,
    )

x_train, x_valid, y_train, y_valid = split_data(padded_features, padded_target)

In [ ]:
print(len(x_train), len(x_train[0]), len(x_train[0][0]))

In [ ]:
OUTPUT_SIZE = len(alphabet) + 1

N_BLSTM_LAYERS = 5
N_CELLS = 64

LEARNING_RATE = 10**-4

BATCH_SIZE = 8
N_FEATURES = len(x_train[0][0])
N_TIMESTEPS = len(x_train[0])
N_EPOCHS = 10

In [ ]:
from keras.layers import Bidirectional, LSTM, Dense, Input, Masking, TimeDistributed

def build_model(n_blstm_layers, n_cells, n_features, output_size):
    model = tf.keras.models.Sequential()

    model.add(Masking(input_shape=(None, N_FEATURES), mask_value=np.zeros((N_FEATURES))))
    
    for i in range(n_blstm_layers):
        model.add(Bidirectional(LSTM(N_CELLS,
                                     input_shape=(None, N_FEATURES),
                                     return_sequences = True,
                                     dropout = 0.5),
                                merge_mode = 'sum'))

    model.add(
        TimeDistributed(
            Dense(output_size, activation = 'softmax')
        )
    )

    return model

In [ ]:
def ctc_loss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

In [ ]:
model = build_model(N_BLSTM_LAYERS, N_CELLS, N_FEATURES, OUTPUT_SIZE)

model.compile(
    loss=ctc_loss,
    optimizer= tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, clipnorm=9)
)

In [ ]:
def create_dataset(train_data, valid_data, n_epochs, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((train_data, valid_data))
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(1)
    return dataset

train_dataset = create_dataset(x_train, y_train, N_EPOCHS, BATCH_SIZE)
validation_dataset = create_dataset(x_valid, y_valid, N_EPOCHS, BATCH_SIZE)

In [ ]:
from pathlib import Path
import pickle

loss_history = [[], []]

loss_path = Path("../../training/logs/encoder_bezier/loss")
if loss_path.is_file():
    with open(loss_path, "rb") as f:
        loss_history = pickle.load(f)

In [ ]:
import keras

checkpoint_path = "../../training/checkpoints/encoder_bezier/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

model.load_weights(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

history = model.fit(
    train_dataset,
    validation_data = validation_dataset,
    shuffle=True,
    epochs = N_EPOCHS,
    batch_size = BATCH_SIZE,
    callbacks = [cp_callback]
)

In [ ]:
loss_history[0].extend(history.history['loss'])
loss_history[1].extend(history.history['val_loss'])

In [ ]:
with open(loss_path, "wb") as f:
    pickle.dump(loss_history, f)
    
print(len(loss_history[0]))

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt

def flatten_list(l):
     return [point for elem in l for point in elem]

plt.plot(range(1000, 1000 + len(loss_history[0])), loss_history[0], color="red", label="train")
plt.plot(range(1000, 1000 + len(loss_history[0])), loss_history[1], color="black", label="test")
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
import sys
sys.path.insert(0, "../autowrite")
from Visualizer import Visualizer

visualizer = Visualizer()
visualizer.plot_bezier_curves(padded_features[10])

In [ ]:
print(len(loss_history[0]))

In [ ]:
from pathlib import Path
if Path("./../../data/processed_data/alphabet").is_file():
    with open("./../../data/processed_data/alphabet", "rb") as f:
        alphabet = pickle.load(f)

print(alphabet, len(alphabet))

def encode_textline(textline, alphabet):
    return [alphabet.index(c) for c in textline]

def decode_textline(encodedline, alphabet):
    return [alphabet[int(v)] if int(v) < len(alphabet) else "blank"for v in encodedline]

def remove_blanks(l):
    return [elem for elem in l if elem != "blank"]

In [ ]:
from Preprocessor import Preprocessor
preprocessor = Preprocessor("../../data/processed_data/alphabet")

path = "./input_example.npy"
raw_strokes = np.load(path, allow_pickle=True)
p = preprocessor.strokes_to_bezier(raw_strokes, precision=0.01)

In [ ]:
sample = p
output = model.call(tf.convert_to_tensor(np.expand_dims(sample, 0)), mask=None)
print(output.shape)
res = output.numpy()[0]

In [ ]:
argmax = np.argmax(res, axis=1)

In [ ]:
import matplotlib.pyplot as plt

print("".join(remove_blanks(decode_textline(padded_target[i], alphabet))))

print("".join(remove_blanks(decode_textline(argmax, alphabet))))

visualizer.plot_bezier_curves(sample)

In [ ]:
reshaped_output = tf.reshape(output, (output.shape[1], output.shape[0], output.shape[2]))
(decoded, log_probabilities) = tf.nn.ctc_beam_search_decoder(reshaped_output, [reshaped_output.shape[0]], beam_width=3)

In [ ]:
print(decode_textline(decoded[0].values, alphabet))

In [ ]:
import kenlm
from pyctcdecode import build_ctcdecoder

# load unigram list
with open("librispeech-vocab.txt") as f:
    unigram_list = [t for t in f.read().strip().split("\n")]

decoder = build_ctcdecoder(
    alphabet,
)
text = decoder.decode(res)
print(text)